# Cursus: Automatic SageMaker (MODS) Pipeline Compiler

The main contribution of this work is **Cursus**, a **compiler** that automatically generate **[MODS (Model Training Workflow Operation and Development System) Pipeline](https://w.amazon.com/bin/view/CMLS/Overview/MODS/)** base on two set of user inputs
* The **Pipeline DAG (Directed Acylic Graph)**, which describe pipeline as a graph
* The **Unified Config JSON**, which provides a central hub to extract all user inputs and their associated step information
    * Run [demo_config](./demo_config.ipynb) first to generate the Unified Config JSON
    * The config json will be saved in `./pipeling_config/xxx/` folder

![mods_pipeline_train_eval_calib](./demo/mods_pipeline_train_eval_calib.png)


In [1]:
import os
import json
import pandas as pd
import pickle
import sys
import subprocess
from datetime import datetime

from pathlib import Path

In [2]:
from pydantic import BaseModel, Field, model_validator, field_validator
from typing import List, Optional, Dict, Any, Type, Union, Tuple

In [3]:
from collections import defaultdict, deque

In [4]:
import logging

In [5]:
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

## Environment Setup

In [6]:
from sagemaker import Session
from secure_ai_sandbox_python_lib.session import Session as SaisSession

2025-12-03 03:48:07,961 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


2025-12-03 03:48:08,362 - INFO - CA certs are provided via the AmazonCACerts installation at /home/ec2-user/.local/lib/python3.10/site-packages/amazoncerts


In [7]:
from mods_workflow_helper.utils.secure_session import create_secure_session_config
from mods_workflow_helper.sagemaker_pipeline_helper import SecurityConfig

from sagemaker.workflow.pipeline_context import PipelineSession

In [8]:
# Initialize session with team bucket
sais_session = SaisSession(".")

security_config = SecurityConfig(
    kms_key=sais_session.get_team_owned_bucket_kms_key(),
    security_group=sais_session.sandbox_vpc_security_group(),
    vpc_subnets=sais_session.sandbox_vpc_subnets(),
)

2025-12-03 03:48:08,964 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2025-12-03 03:48:09,762 - INFO - successfully patched module botocore


In [9]:
sagemaker_config = create_secure_session_config(
    role_arn=PipelineSession().get_caller_identity_arn(),
    # If you are uploading to andes, use cradle_read_s3_bucket_name() and get_cradle_read_bucket_kms_key() respecitely
    bucket_name=sais_session.team_owned_s3_bucket_name(),
    kms_key=sais_session.get_team_owned_bucket_kms_key(),
    vpc_subnet_ids=sais_session.sandbox_vpc_subnets(),
    vpc_security_groups=[sais_session.sandbox_vpc_security_group()],
)

2025-12-03 03:48:09,783 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2025-12-03 03:48:10,143 - INFO - There is no MODS workflow execution id provided, this is probably because you are running your pipeline outside of MODS.


In [10]:
pipeline_session = PipelineSession(
    default_bucket=sais_session.team_owned_s3_bucket_name(),
    sagemaker_config=sagemaker_config,
)  # IMPORTANT now the session uses the generated sagemaker_config

2025-12-03 03:48:10,163 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [11]:
pipeline_session.config = sagemaker_config

In [12]:
bucket = sais_session.team_owned_s3_bucket_name()
bucket

'sandboxdependency-abuse-secureaisandboxteamshare-1l77v9am252um'

In [13]:
role = PipelineSession().get_caller_identity_arn()
role

2025-12-03 03:48:10,711 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


'arn:aws:iam::601857636239:role/SandboxRole-lukexie-us-east-1'

In [14]:
from pathlib import Path
import sys

# Get parent directory of current notebook
project_root = str(Path().absolute().parent)
print(f"project root {project_root}")
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"add project root {project_root} into system")

project root /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template
add project root /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template into system


## Basic Information

In [15]:
region_list = ["NA", "EU", "FE"]

In [16]:
region_selection = 0

In [17]:
region = region_list[region_selection]
region

'NA'

In [18]:
MODEL_CLASS = "pytorch"

In [19]:
service_name = "BuyerAbuseRnR"

#### Config and Hyperparameter Information

In [20]:
current_dir = Path.cwd()
config_dir = Path(current_dir) / "pipeline_config"
print(config_dir)

/home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/pipeline_config


In [21]:
# hyparam_filename = f'hyperparameters_{region}_{MODEL_CLASS}.json' #'hyperparameters.json'

In [22]:
pipeline_config_name = f"config.json"  # f'config_{region}.json'
pipeline_config_name

'config.json'

In [23]:
config_path = config_dir / pipeline_config_name

In [24]:
config_path

PosixPath('/home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/pipeline_config/config.json')

## Pipeline Imports

In [25]:
from enum import Enum
from pydantic import BaseModel

## [Optional]: Test Config Load Functionality

Please skip this section if you are not concern about the config information loaded

### Hyperparameters

In [26]:
# from cursus.steps.hyperparams.hyperparameters_xgboost import XGBoostModelHyperparameters

In [27]:
# hyparam_path = config_dir / hyparam_filename
# with open(hyparam_path, 'r') as file:
#    hyperparam_dict = json.load(file)

In [28]:
# hyperparams = XGBoostModelHyperparameters(**hyperparam_dict)

In [29]:
# hyperparams.num_classes

In [30]:
# hyperparams.is_binary

### Import Configs

In [31]:
from cursus.core.base.config_base import BasePipelineConfig

In [32]:
# from cursus.steps.configs.config_cradle_data_loading_step import (CradleDataLoadingConfig,
#                                                    MdsDataSourceConfig,
#                                                    EdxDataSourceConfig,
#                                                    DataSourceConfig,
#                                                    DataSourcesSpecificationConfig,
#                                                    JobSplitOptionsConfig,
#                                                    TransformSpecificationConfig,
#                                                    OutputSpecificationConfig,
#                                                    CradleJobSpecificationConfig
#                                                   )

In [33]:
from cursus.steps.configs.config_dummy_data_loading_step import DummyDataLoadingConfig
from cursus.steps.configs.config_tabular_preprocessing_step import (
    TabularPreprocessingConfig,
)
from cursus.steps.configs.config_bedrock_prompt_template_generation_step import (
    BedrockPromptTemplateGenerationConfig,
)
from cursus.steps.configs.config_bedrock_batch_processing_step import (
    BedrockBatchProcessingConfig,
)
from cursus.steps.configs.config_label_ruleset_generation_step import (
    LabelRulesetGenerationConfig,
)
from cursus.steps.configs.config_label_ruleset_execution_step import (
    LabelRulesetExecutionConfig,
)
from cursus.steps.configs.config_pytorch_training_step import PyTorchTrainingConfig
from cursus.steps.configs.config_pytorch_model_eval_step import PyTorchModelEvalConfig
from cursus.steps.configs.config_dummy_training_step import DummyTrainingConfig
from cursus.steps.configs.config_model_calibration_step import ModelCalibrationConfig
from cursus.steps.configs.config_package_step import PackageConfig
from cursus.steps.configs.config_payload_step import PayloadConfig
from cursus.steps.configs.config_registration_step import RegistrationConfig

### Load Config

In [34]:
from cursus.steps.configs.utils import (
    serialize_config,
    merge_and_save_configs,
    load_configs,
    verify_configs,
)

In [35]:
CONFIG_CLASSES = {
    "DummyDataLoadingConfig": DummyDataLoadingConfig,
    "BedrockPromptTemplateGenerationConfig": BedrockPromptTemplateGenerationConfig,
    "BedrockBatchProcessingConfig": BedrockBatchProcessingConfig,
    "TabularPreprocessingConfig": TabularPreprocessingConfig,
    "LabelRulesetGenerationConfig": LabelRulesetGenerationConfig,
    "LabelRulesetExecutionConfig": LabelRulesetExecutionConfig,
    "PyTorchTrainingConfig": PyTorchTrainingConfig,
    "PyTorchModelEvalConfig": PyTorchModelEvalConfig,
    "DummyTrainingConfig": DummyTrainingConfig,
    "ModelCalibrationConfig": ModelCalibrationConfig,
    "PackageConfig": PackageConfig,
    "PayloadConfig": PayloadConfig,
    "RegistrationConfig": RegistrationConfig,
}

In [36]:
config_path

PosixPath('/home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/pipeline_config/config.json')

In [37]:
# Load configs
loaded_configs = load_configs(config_path, CONFIG_CLASSES)

2025-12-03 03:48:11,761 - INFO - Loading configs from /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/pipeline_config/config.json
2025-12-03 03:48:11,761 - INFO - Loading configuration from /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/pipeline_config/config.json
2025-12-03 03:48:11,762 - INFO - Successfully loaded configuration from /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/pipeline_config/config.json
2025-12-03 03:48:11,763 - INFO - Successfully loaded configs from /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/pipeline_config/config.json with 8 specific configs
2025-12-03 03:48:11,764 - INFO - Creating additional config instance for DummyDataLoading_calibration (DummyDataLoadingConfig)
2025-12-03 03:48:11,764 - INFO - 🔧 BuilderAutoDiscovery.__init__ starting - package_root: /ho

In [38]:
loaded_configs

{'DummyDataLoading_calibration': DummyDataLoadingConfig(author='lukexie', bucket='sandboxdependency-abuse-secureaisandboxteamshare-1l77v9am252um', role='arn:aws:iam::601857636239:role/SandboxRole-lukexie-us-east-1', region='NA', service_name='BuyerAbuseRnR', pipeline_version='0.0.2', model_class='pytorch', current_date='2025-12-03', framework_version='2.1.0', py_version='py310', source_dir='dockers', enable_caching=False, use_secure_pypi=False, max_runtime_seconds=172800, project_root_folder='rnr_pytorch_bedrock', processing_instance_count=1, processing_volume_size=500, processing_instance_type_large='ml.m5.12xlarge', processing_instance_type_small='ml.m5.4xlarge', use_large_processing_instance=True, processing_source_dir='dockers/scripts', processing_entry_point='dummy_data_loading.py', processing_script_arguments=None, processing_framework_version='1.2-1', data_source='/home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/processed_data/tes

In [39]:
len(loaded_configs)

8

In [40]:
[str(k) for k in loaded_configs.keys()]

['DummyDataLoading_calibration',
 'DummyTraining',
 'ModelCalibration_calibration',
 'Package',
 'Payload',
 'PyTorchModelEval_calibration',
 'Registration',
 'TabularPreprocessing_calibration']

In [41]:
first_config = next(iter(loaded_configs.values()))

In [42]:
PIPELINE_VERSION = first_config.pipeline_version

In [43]:
PIPELINE_DESCRIPTION = first_config.pipeline_description

In [44]:
PIPELINE_NAME = first_config.pipeline_name

## Parameter Setup

In [45]:
from mods_workflow_core.utils.constants import (
    PIPELINE_EXECUTION_TEMP_DIR,
    KMS_ENCRYPTION_KEY_PARAM,
    PROCESSING_JOB_SHARED_NETWORK_CONFIG,
    SECURITY_GROUP_ID,
    VPC_SUBNET,
)

## Import Packages

In [46]:
from abc import ABC, abstractmethod
from typing import Dict, List, Any, Optional, Type
from pathlib import Path
import logging
import os
import importlib

In [47]:
import sagemaker
from sagemaker import Session
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.parameters import Parameter
from sagemaker.workflow.properties import Properties
from sagemaker.workflow.pipeline_context import PipelineSession  # Crucial import

## Demo: An End-to-End Pipeline based on PipelineDAG Compiler
Let us use the following simpler DAG (without registration as example)


In this demo there are several user input
* the **Unified JSON file** in `config_path`
* the **Registry Manager**: an object that handles the map between step logical name to `step.properties`
* the **Dependency Resolver**: an object than handles the *automatic dependency resolution* between steps
* the other fields
    * `sagemaker_session`: pipelne session
    * `role`: IAM Role
    * `notebook_root`: track the root path 


In this pipeline template, we inherit from base class `PipelineTemplateBase`. 

The **major tasks** are
* *`Config` Classes Import*
* *Configuration Validation*
* *Step Builder Retrieval and Step Builder Map Creation*
* *Configuration Map Creation*
* **Pipeline DAG Generation**: ideally, user should create this DAG and use it as input
* **Automatic Pipeline Assemble**: Call `pipeline_assembler`


### DAG to Template Compiler

In [48]:
from cursus.api.dag.base_dag import PipelineDAG
from cursus.core.compiler.dag_compiler import (
    compile_dag_to_pipeline,
    PipelineDAGCompiler,
)
from cursus.core.compiler.validation import ConversionReport
from cursus.steps.configs.utils import load_configs

In [49]:
def create_bedrock_batch_pytorch_with_label_ruleset_e2e_dag() -> PipelineDAG:
    """
    Create a complete end-to-end XGBoost pipeline DAG.

    This DAG represents the same workflow as the legacy demo_config.ipynb
    but in a structured, reusable format.

    Returns:
        PipelineDAG: The directed acyclic graph for the pipeline
    """
    dag = PipelineDAG()

    # Add all nodes - incorporating Bedrock batch processing and label ruleset steps
    dag.add_node("DummyTraining")  # Dummy data load for training
    dag.add_node(
        "ModelCalibration_calibration"
    )  # Model calibration step with calibration variant
    dag.add_node("Package")  # Package step
    dag.add_node("Registration")  # MIMS registration step
    dag.add_node("Payload")  # Payload step
    dag.add_node("DummyDataLoading_calibration")  # Dummy data load for calibration
    dag.add_node(
        "TabularPreprocessing_calibration"
    )  # Tabular preprocessing for calibration
    dag.add_node("PyTorchModelEval_calibration")  # Model evaluation step

    # Calibration flow with Bedrock batch processing and label ruleset integration
    dag.add_edge("DummyDataLoading_calibration", "TabularPreprocessing_calibration")

    # Evaluation flow
    dag.add_edge("DummyTraining", "PyTorchModelEval_calibration")
    dag.add_edge(
        "TabularPreprocessing_calibration", "PyTorchModelEval_calibration"
    )  # Use labeled calibration data

    # Model calibration flow - depends on model evaluation
    dag.add_edge("PyTorchModelEval_calibration", "ModelCalibration_calibration")

    # Output flow
    dag.add_edge("ModelCalibration_calibration", "Package")
    dag.add_edge("DummyTraining", "Package")  # Raw model is also input to packaging
    dag.add_edge("DummyTraining", "Payload")  # Payload test uses the raw model
    dag.add_edge("Package", "Registration")
    dag.add_edge("Payload", "Registration")

    logger.info(
        f"Created XGBoost E2E DAG with {len(dag.nodes)} nodes and {len(dag.edges)} edges"
    )
    return dag

In [50]:
dag = create_bedrock_batch_pytorch_with_label_ruleset_e2e_dag()

2025-12-03 03:48:12,000 - INFO - Added node: DummyTraining
2025-12-03 03:48:12,001 - INFO - Added node: ModelCalibration_calibration
2025-12-03 03:48:12,001 - INFO - Added node: Package
2025-12-03 03:48:12,001 - INFO - Added node: Registration
2025-12-03 03:48:12,002 - INFO - Added node: Payload
2025-12-03 03:48:12,002 - INFO - Added node: DummyDataLoading_calibration
2025-12-03 03:48:12,002 - INFO - Added node: TabularPreprocessing_calibration
2025-12-03 03:48:12,003 - INFO - Added node: PyTorchModelEval_calibration
2025-12-03 03:48:12,003 - INFO - Added edge: DummyDataLoading_calibration -> TabularPreprocessing_calibration
2025-12-03 03:48:12,003 - INFO - Added edge: DummyTraining -> PyTorchModelEval_calibration
2025-12-03 03:48:12,004 - INFO - Added edge: TabularPreprocessing_calibration -> PyTorchModelEval_calibration
2025-12-03 03:48:12,004 - INFO - Added edge: PyTorchModelEval_calibration -> ModelCalibration_calibration
2025-12-03 03:48:12,004 - INFO - Added edge: ModelCalibratio

In [51]:
pipeline_parameters = [
    PIPELINE_EXECUTION_TEMP_DIR,
    KMS_ENCRYPTION_KEY_PARAM,
    SECURITY_GROUP_ID,
    VPC_SUBNET,
]

In [52]:
dag_compiler = PipelineDAGCompiler(
    config_path=config_path,
    sagemaker_session=pipeline_session,
    role=role,
    pipeline_parameters=pipeline_parameters,
)

2025-12-03 03:48:12,017 - INFO - 🔧 BuilderAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/cursus
2025-12-03 03:48:12,017 - INFO - 🔧 BuilderAutoDiscovery.__init__ - workspace_dirs: []
2025-12-03 03:48:12,018 - INFO - ✅ BuilderAutoDiscovery basic initialization complete
2025-12-03 03:48:12,018 - INFO - ✅ Registry info loaded: 43 steps
2025-12-03 03:48:12,019 - INFO - 🎉 BuilderAutoDiscovery initialization completed successfully
2025-12-03 03:48:12,019 - INFO - 🔍 ScriptAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/cursus
2025-12-03 03:48:12,019 - INFO - 🔍 ScriptAutoDiscovery.__init__ - workspace_dirs: []
2025-12-03 03:48:12,020 - INFO - 🔍 ScriptAutoDiscovery.__init__ - priority_workspace_dir: None
2025-12-03 03:48:12,021 - INFO - ✅ Registry info loaded: 43 steps
2025-12-03 03:48:12,021 - INFO - 🎉 ScriptAutoDiscovery initialization com

### Create a Pipeline

#### DAG Validation and Preview of Config Resolution

In [53]:
preview_only = True

In [54]:
if preview_only:
    preview = dag_compiler.preview_resolution(dag)
    logger.info("DAG node resolution preview:")
    for node, config_type in preview.node_config_map.items():
        confidence = preview.resolution_confidence.get(node, 0.0)
        logger.info(f"  {node} → {config_type} (confidence: {confidence:.2f})")

    if preview.recommendations:
        logger.info("Recommendations:")
        for recommendation in preview.recommendations:
            logger.info(f"  - {recommendation}")

    validation = dag_compiler.validate_dag_compatibility(dag)
    logger.info(f"DAG validation: {'VALID' if validation.is_valid else 'INVALID'}")
    if not validation.is_valid:
        if validation.missing_configs:
            logger.warning(f"Missing configs: {validation.missing_configs}")
        if validation.unresolvable_builders:
            logger.warning(f"Unresolvable builders: {validation.unresolvable_builders}")
        if validation.config_errors:
            logger.warning(f"Config errors: {validation.config_errors}")

2025-12-03 03:48:12,035 - INFO - Previewing resolution for 8 DAG nodes
2025-12-03 03:48:12,035 - INFO - Creating template for DAG with 8 nodes
2025-12-03 03:48:12,035 - WARNING - Could not import config_class_detector, using fallback implementation
2025-12-03 03:48:12,036 - INFO - 🔧 BuilderAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/cursus
2025-12-03 03:48:12,036 - INFO - 🔧 BuilderAutoDiscovery.__init__ - workspace_dirs: []
2025-12-03 03:48:12,037 - INFO - ✅ BuilderAutoDiscovery basic initialization complete
2025-12-03 03:48:12,037 - INFO - ✅ Registry info loaded: 43 steps
2025-12-03 03:48:12,038 - INFO - 🎉 BuilderAutoDiscovery initialization completed successfully
2025-12-03 03:48:12,038 - INFO - 🔍 ScriptAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/cursus
2025-12-03 03:48:12,039 - INFO - 🔍 ScriptAutoDiscovery.__init__ - work

### Put it Together: Pipeline Generation from DAG

In [ ]:
# Convert DAG to pipeline and get report
try:
    logger.info(f"Converting DAG to pipeline")
    template_pipeline, report = dag_compiler.compile_with_report(dag=dag)

    # Log report summary
    logger.info(f"Conversion complete: {report.summary()}")
    for node, details in report.resolution_details.items():
        logger.info(f"  {node} → {details['config_type']} ({details['builder_type']})")

    # Log pipeline creation details
    logger.info(f"Pipeline '{template_pipeline.name}' created successfully")
    logger.info(
        f"Pipeline ARN: {template_pipeline.arn if hasattr(template_pipeline, 'arn') else 'Not available until upserted'}"
    )
    logger.info("To upsert the pipeline, call pipeline.upsert()")
except Exception as e:
    logger.error(f"Failed to convert DAG to pipeline: {e}")
    raise

2025-12-03 03:48:12,187 - INFO - Converting DAG to pipeline
2025-12-03 03:48:12,188 - INFO - Compiling DAG with detailed reporting
2025-12-03 03:48:12,188 - INFO - Compiling DAG with 8 nodes to pipeline
2025-12-03 03:48:12,189 - INFO - Creating template for DAG with 8 nodes
2025-12-03 03:48:12,189 - INFO - Loading configs from: /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/pipeline_config/config.json
2025-12-03 03:48:12,190 - INFO - 🔧 BuilderAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/cursus
2025-12-03 03:48:12,190 - INFO - 🔧 BuilderAutoDiscovery.__init__ - workspace_dirs: []
2025-12-03 03:48:12,191 - INFO - ✅ BuilderAutoDiscovery basic initialization complete
2025-12-03 03:48:12,191 - INFO - ✅ Registry info loaded: 43 steps
2025-12-03 03:48:12,191 - INFO - 🎉 BuilderAutoDiscovery initialization completed successfully
2025-12-03 03:48:12,192 - INFO - 🔍 Sc

sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.ProcessingOutputConfig.KmsKeyId
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.ProcessingResources.ClusterConfig.VolumeKmsKeyId
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.SecurityGroupIds
sagemaker.config INFO - Applied value from config key = SageMaker.TrainingJob.ResourceConfig.VolumeKmsKeyId
sagemaker.config INFO - Applied value from config key = SageMaker.TrainingJob.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.TrainingJob.VpcConfig.SecurityGroupIds
sagemaker.config INFO - Applied value from config key = SageMaker.TrainingJob.Environment


2025-12-03 03:48:14,548 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2025-12-03 03:48:14,920 - INFO - [Tier 1 - Config] Using pretrained_model_path: s3://sandboxdependency-abuse-secureaisandboxteamshare-1l77v9am252um/lukexie-BuyerAbuseRnR-pytorch-NA-1-0-0-pipeline-2025-12-02-07-09-42/pytorch_training/pipelines-uu186yt9zlqz-PyTorchTraining-1aiejj5VXs/output/model.tar.gz
2025-12-03 03:48:14,921 - INFO - [SOURCE Fallback] No hyperparameters input - using code directory or source_dir/hyperparams/
2025-12-03 03:48:14,921 - INFO - Using execution_prefix for base output path
2025-12-03 03:48:14,922 - INFO - Processing PipelineVariable for output_path: {...}
2025-12-03 03:48:14,922 - INFO - No command-line arguments needed for dummy training script
2025-12-03 03:48:14,922 - INFO - Using resolved script path: /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/dockers/scripts/dummy_training.py
2025-12-03 03:48:14,923

sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.ProcessingOutputConfig.KmsKeyId
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.ProcessingResources.ClusterConfig.VolumeKmsKeyId
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.SecurityGroupIds


2025-12-03 03:48:15,962 - INFO - Using user-provided data source from configuration: /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/processed_data/test -> /opt/ml/processing/input/data
2025-12-03 03:48:15,962 - INFO - No command-line arguments needed for dummy data loading script
2025-12-03 03:48:15,963 - INFO - Using script path: /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/dockers/scripts/dummy_data_loading.py
2025-12-03 03:48:15,964 - INFO - Created ProcessingStep with name: DummyDataLoading-Calibration
2025-12-03 03:48:15,964 - INFO - Built step DummyDataLoading_calibration
2025-12-03 03:48:15,965 - INFO - Using execution_prefix for base output path
2025-12-03 03:48:15,965 - INFO - Creating MIMS Payload ProcessingStep...
2025-12-03 03:48:15,966 - INFO - Registered specification for step 'PayloadStepStep' of type 'Payload' in context 'lukexie-BuyerAbuseRnR-pytorch-NA'
2025-12-03 0

sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.ProcessingOutputConfig.KmsKeyId
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.ProcessingResources.ClusterConfig.VolumeKmsKeyId
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.SecurityGroupIds


2025-12-03 03:48:17,003 - INFO - No command-line arguments needed for payload script
2025-12-03 03:48:17,003 - INFO - Using script path: /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/rnr_pytorch_bedrock/dockers/scripts/payload.py
2025-12-03 03:48:17,004 - INFO - Created ProcessingStep with name: Payload
2025-12-03 03:48:17,005 - INFO - Built step Payload
2025-12-03 03:48:17,005 - INFO - Using execution_prefix for base output path
2025-12-03 03:48:17,005 - INFO - Registered specification for step 'TabularPreprocessingStepStep' of type 'TabularPreprocessing' in context 'lukexie-BuyerAbuseRnR-pytorch-NA'
2025-12-03 03:48:17,006 - INFO - Registered specification for step 'DummyDataLoading-Calibration' of type 'DummyDataLoading' in context 'lukexie-BuyerAbuseRnR-pytorch-NA'
2025-12-03 03:48:17,007 - INFO - Best match for DATA: DummyDataLoading-Calibration.DATA (confidence: 1.000)
2025-12-03 03:48:17,007 - INFO - Resolved TabularPreprocessingStepStep.DATA -> D

sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.ProcessingOutputConfig.KmsKeyId
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.ProcessingResources.ClusterConfig.VolumeKmsKeyId
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.SecurityGroupIds


### Pipeline Template

After the pipeline is generated, we can retrieve the pipeline template

In [ ]:
pipeline_template_builder = dag_compiler.get_last_template()

## Start Execution

## Start Execution

In [ ]:
role_arn = pipeline_session.get_caller_identity_arn()
role_arn

In [ ]:
pipeline_description = PIPELINE_DESCRIPTION

In [ ]:
PIPELINE_DESCRIPTION

## Prepare for Execution Document

In [ ]:
from mods_workflow_helper.sagemaker_pipeline_helper import (
    SagemakerPipelineHelper,
    SecurityConfig,
)

In [ ]:
default_execution_doc = SagemakerPipelineHelper.get_pipeline_default_execution_document(
    template_pipeline
)
test_execution_doc = default_execution_doc

In [ ]:
print(json.dumps(test_execution_doc, indent=2))

In [ ]:
# with open(config_dir / 'sample_exe_doc.json', 'w') as f:
#    json.dump(default_execution_doc, f, indent=2)

### Fill in Execution Doc

In [ ]:
from cursus.mods.exe_doc.generator import ExecutionDocumentGenerator

In [ ]:
exe_doc_generator = ExecutionDocumentGenerator(
    config_path=config_path,
    sagemaker_session=pipeline_session,
    role=role,
)

In [ ]:
execution_doc_fill = exe_doc_generator.fill_execution_document(
    dag=dag, execution_document=test_execution_doc
)

In [ ]:
# execution_doc_fill = pipeline_builder.fill_execution_document(test_execution_doc)

In [ ]:
# Fill in the execution document using the stored requests
# execution_doc_fill_2 = xgboost_train_eval_pipeline_template_builder.fill_execution_document(test_execution_doc)

In [ ]:
print(json.dumps(execution_doc_fill, indent=2))

In [ ]:
test_execution_doc = execution_doc_fill.copy()

### Save Execution Doc locally

In [ ]:
PIPELINE_NAME = first_config.pipeline_name

In [ ]:
PIPELINE_VERSION = first_config.pipeline_version

In [ ]:
exe_doc_json_filename = f"execute_doc_{PIPELINE_NAME}_{PIPELINE_VERSION}.json"
exe_doc_file_path = config_dir / exe_doc_json_filename
exe_doc_file_path

In [ ]:
with open(exe_doc_file_path, "w") as f:
    json.dump(test_execution_doc, f, indent=2)

## Execute Pipeline

### Start Execution

In [ ]:
from mods_workflow_helper.sagemaker_pipeline_helper import SagemakerPipelineHelper

In [ ]:
security_config

In [ ]:
template_pipeline

In [ ]:
SagemakerPipelineHelper.start_pipeline_execution(
    pipeline=template_pipeline,
    secure_config=security_config,
    sagemaker_session=pipeline_session,
    preparation_space_local_root="/tmp",
    pipeline_execution_document=test_execution_doc,
)